In [1]:
import importlib
from lib import kpi

In [42]:
importlib.reload(kpi)

<module 'lib.kpi' from 'c:\\workspace\\GitHub\\arcsaef\\lib\\kpi.py'>

In [2]:
data           = kpi.load_data()
rpt_config     = kpi.get_rpt_args()
saef_library   = kpi.get_saef_library()
responses_json = kpi.split_response(data[0])
ppl_collection = kpi.person_construct(responses_json, data[2], rpt_config[0])
proj_saef      = kpi.project_construct(responses_json, ppl_collection[0])
buckets        = kpi.matched_library(saef_library, ppl_collection[1])
templates      = kpi.load_templates()
meta_bucket    = buckets[0]
bucket         = buckets[1]
ppl_saef       = dict(sorted(ppl_collection[0].items(), key = lambda x: x[1].get('LastName')))
ppl_hash       = ppl_collection[1]
bulk_responses = data[0]
biblio         = data[1]
scopus         = data[2]
rpt_year       = rpt_config[0]
organisations  = rpt_config[1]
org_shortnames = list(organisations.keys())
proj_saef_nohold = proj_saef[proj_saef.Status != 'On hold']

In [3]:
# 1.remove empty rows
bucket = bucket[bucket['id_person'].isna() == False]
# 2. Make parsing more straight forward
for prsn in ppl_saef:
    if ppl_saef[prsn]['Gender'] == 'Non-binary/Gender diverse' or  ppl_saef[prsn]['Gender']  == 'Prefer not to say':
        ppl_saef[prsn]['Gender'] = 'Other'
# 3. Exclude: Nicole Webster, 027E2DEA-DB06-3946-B8C8-E053EF8E09F0
del ppl_saef['027E2DEA-DB06-3946-B8C8-E053EF8E09F0']; del ppl_hash['NicoleWebster']
# 4. Remove leading/trialing whitespace from  blibliographic entry.
biblio['Biblio'] = biblio['Biblio'].str.strip()
# 5. Jump into your pedantry
for prsn in ppl_saef:
    if ppl_saef[prsn]['Position'] != 'PhD Student' and  ppl_saef[prsn]['Position'] != 'Masters Student' and \
       ppl_saef[prsn]['Position'] != 'Honours Student':
        ppl_saef[prsn]['StudentProjectTitle'] = 'Not applicable'

 ### A brief what's where and why
 - Reporting data is from 3 sources;
   - Filemaker: api output (via Postman) is saved as text
     - data/kpi_reporting_yyyymmdd.txt
   - Zotero: api output for reporting year (via Postman) is saved as json
     - data/saef_library_postman_yyyymmdd.json
   - Bibiliography: Zotero bibligraphy output & manual update is saved a xlsx
     - data/all/zot_biblio.xlsx
 - Updating config/reporting.yaml is critical
 - Individual and Organisation cells are run for mid-year and annual reporting
 - Organisation Excel is uused for mid-year

In [43]:
kpi.write_pub_auth_excel(bucket, ppl_saef)

In [37]:
bucket

,key,itemType,title,rights,pubyr,name,id_person,tags,publicationTitle,project
0,E7534TV4,journalArticle,The atmosphere: a transport medium or an active microbial ecosystem?,oa,2024,RachaelLappan,6BC16DD4-3E26-AB41-8A1A-09F47E0C24F3,[],The ISME Journal,
2,JFHT2C9T,journalArticle,Including a diverse set of voices to address biological invasions,,2024,MelodieMcGeoch,9346D92C-9533-A94C-98DB-4C7F05B79FCD,[],Trends in Ecology & Evolution,
3,6RBB7SDM,dataset,Parsayarya/Scraping-ATCM: ATCM WP and IP paper dataset,Creative Commons Attribution 4.0 International,2024,MichaelBode,28E905A2-8596-F640-84FF-21A92FA343C7,[],None,
4,LLDRRG65,dataset,Parsayarya/ATS-Measure-Scraping: ATS Measure Dataset,Creative Commons Attribution 4.0 International,2024,MichaelBode,28E905A2-8596-F640-84FF-21A92FA343C7,[],None,
5,4XYRH4X6,journalArticle,Is tree planting an effective strategy for climate change mitigation?,12mo,2024,SharonRobinson,B3BB1CEF-1CDF-BD42-BC0A-21A212CCAB1A,[],Science of The Total Environment,
6,GB66JQII,journalArticle,Macroevolution of the plant-hummingbird pollination system,oa,2024,MatthiasDehling,0641E93F-1887-994C-86FE-510E18546AFC,[],Biological Reviews of the Cambridge Philosophical Society,
8,CEC442BP,journalArticle,On the relationship between mesoscale cellular convection and meteorological forcing: comparing the Southern Ocean against the North Pacific,oa,2024,StevenSiems,C1B89E8D-7298-5B41-8567-946D0A56323B,[],Atmospheric Chemistry and Physics,
9,CEC442BP,journalArticle,On the relationship between mesoscale cellular convection and meteorological forcing: comparing the Southern Ocean against the North Pacific,oa,2024,Tahereh(Nasim)Alinejadtabrizi,2947F044-F5D0-EE47-B913-296B314FEEC8,[],Atmospheric Chemistry and Physics,
10,6STGMD7P,journalArticle,Wet deposition in shallow convection over the Southern Ocean,oa,2024,StevenSiems,C1B89E8D-7298-5B41-8567-946D0A56323B,[],npj Climate and Atmospheric Science,
11,6STGMD7P,journalArticle,Wet deposition in shallow convection over the Southern Ocean,oa,2024,Tahereh(Nasim)Alinejadtabrizi,2947F044-F5D0-EE47-B913-296B314FEEC8,[],npj Climate and Atmospheric Science,
